In [1]:
import numpy
import simpy
import random

In [44]:
SERVER1REQUESTMEDIAN=0.7
SERVER2REQUESTMEDIAN=1.0
PSERVER1=0.6
PSERVER2=1-PSERVER1

SERVER3REQUESTMEDIAN=0.8 #MODELO2

REQUESTINTERVALMEDIAN=4.0
REQUESTCOUNT=100000

## MODELO1

In [45]:
def source2(env, number, interval, server1, server2, data):
    """Source generates requests randomly"""
    for i in range(number):
        if random.random()<=PSERVER1:
            c = request(env, 'Request%02d' % i, server1, SERVER1REQUESTMEDIAN, data)
            env.process(c)
            t = random.expovariate(1.0 / interval)
            yield env.timeout(t)
        else:
            c = request(env, 'Request%02d' % i, server2, SERVER2REQUESTMEDIAN, data)
            env.process(c)
            t = random.expovariate(1.0 / interval)
            yield env.timeout(t)
            
def request(env, name, server, requesttime, data):
    """Request arrives, and is resolved"""
    arrive = env.now

    with server.request() as req:
        
        results = yield req

        wait = env.now - arrive
        if wait>0:
            data[0]+=wait
            data[1]+=1
        
        tib = random.expovariate(1.0 / requesttime)
        yield env.timeout(tib)
        total_wait = env.now - arrive
        data[2]+=total_wait
        

In [46]:
env2=simpy.Environment()
server1=simpy.Resource(env2, 1)
server2=simpy.Resource(env2, 1)

data=[0,0,0]

env2.process(source2(env2, REQUESTCOUNT, REQUESTINTERVALMEDIAN, server1, server2, data))
env2.run()

## MODELO2

In [47]:
def source(env, number, interval, server, data):
    """Source generates requests randomly"""
    for i in range(number):
        c = request(env, 'Request%02d' % i, server, SERVER3REQUESTMEDIAN, data)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)


In [48]:
acum_tiempo=0

env=simpy.Environment()
server1=simpy.Resource(env, 1)
server2=simpy.Resource(env, 1)


data2=[0,0,0]

env.process(source(env, REQUESTCOUNT, REQUESTINTERVALMEDIAN, server1, data2))
env.run()

## RESULTADOS MODELO1

In [49]:
print('El tiempo medio de cola de las requests es %7.4f' % (data[0]/float(REQUESTCOUNT)))
print('El porcentaje total de requests que no tuvieron que esperar es %7.4f: ' % (data[1]/float(REQUESTCOUNT)))
print('El tiempo medio de cola mas procesamiento es %7.4f' % (data[2]/float(REQUESTCOUNT)))

El tiempo medio de cola de las requests es  0.0933
El porcentaje total de requests que no tuvieron que esperar es  0.1032: 
El tiempo medio de cola mas procesamiento es  0.9118


## RESULTADOS MODELO2

In [50]:
print('El tiempo medio de cola de las requests es %7.4f' % (data2[0]/float(REQUESTCOUNT)))
print('El porcentaje total de requests que no tuvieron que esperar es %7.4f: ' % (data2[1]/float(REQUESTCOUNT)))
print('El tiempo medio de cola mas procesamiento es %7.4f' % (data2[2]/float(REQUESTCOUNT)))

El tiempo medio de cola de las requests es  0.2044
El porcentaje total de requests que no tuvieron que esperar es  0.2009: 
El tiempo medio de cola mas procesamiento es  1.0062


## CONCLUSION

In [51]:
if data[2]<data2[2]*0.5:
    print('Es conveniente elegir el modelo 1 ya que el tiempo medio para resolver cada solicitud es 50% menor que usando el modelo 2')
else: print('Es conveniente elegir el modelo 2 ya que el tiempo medio para resolver cada solicitud no es 50% menor usando el modelo 1')

Es conveniente elegir el modelo 2 ya que el tiempo medio para resolver cada solicitud no es 50% menor usando el modelo 1
